In [ ]:
#Importing required libraries for data cleanup on CSV files, and to create DataFrames
import pandas as pd
from pathlib import Path
import numpy as np
import csv
from datetime import datetime
import sqlalchemy
import os
import requests
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import hvplot.pandas
from hvplot import hvPlot
import holoviews as hv

In [ ]:
# Load .env environment variables
load_dotenv()

# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Create the Alpaca API object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

In [ ]:
#Creating Dataframes from the CSV files
inlaw_df = pd.read_csv(Path("Updated_CSV/inlaw_csv"), infer_datetime_format=True, parse_dates=True, index_col='Date')
johnson_df = pd.read_csv(Path("Updated_CSV/johnson_csv"), infer_datetime_format=True, parse_dates=True, index_col='Date') 
johnson_ira_df = pd.read_csv(Path("Updated_CSV/johnson_ira_csv"), infer_datetime_format=True, parse_dates=True, index_col='Date')

In [ ]:
#Changing Dates to datetime formats
inlaw_df['Expiration Date'] = pd.to_datetime(inlaw_df['Expiration Date'])
johnson_df['Expiration Date'] = pd.to_datetime(johnson_df['Expiration Date'])
johnson_ira_df['Expiration Date'] = pd.to_datetime(johnson_ira_df['Expiration Date'])

In [ ]:
#Creating SQL Database w/ all dataframes
database_connection_string = 'sqlite:///portfolios.db'
engine = sqlalchemy.create_engine(database_connection_string)

In [ ]:
#creating SQL database w/ dataframes
inlaw_df.to_sql('inlaw', engine, if_exists='replace')
johnson_df.to_sql('johnson', engine, if_exists='replace')
johnson_ira_df.to_sql('johnsonira', engine, if_exists='replace')

In [ ]:
#Narrow down dataframe with SQL Query to pull only data required to calculate returns for pure equity transactions.
query = """
SELECT *
FROM inlaw
WHERE Type = "Trade" AND "Instrument Type" == "Equity";
"""
inlaw_equities_df = pd.read_sql_query(query, con=engine)

In [ ]:
#Convert Average Price to absolute value to remove negative values.
inlaw_equities_df['Average Price'] = inlaw_equities_df['Average Price'].abs()

In [ ]:
#Define function to retrieve total curent position for each Symbol. 
def get_total_equities_position(df):

    #Create variable to get quantity value in each row of the Dataframe
    quantity = df["Quantity"]

    #Creating conditional statements where if there is a Buy Action, then create a new column called "Position" and put in the value of -1
    #Else, set the "Position" to 1
    df.loc[(df['Action'] == "BUY_TO_OPEN"), 'Position'] = -1
    df.loc[(df['Action'] != "SELL_TO_CLOSE"), 'Position'] = 1

    #Create a variable for the actual numerical position of the options trade
    position = df["Position"]

    #Creating a new column in the DataFrame to hold the total position for the row
    df["Total Position"] = quantity * position

    #Getting the total position by summing all values in the Total Position column
    sum_df = df.groupby("Symbol").sum()
    
    return sum_df

In [ ]:
#Define function to get mean value of the columns so we can determine average price paid per share.
def get_avg_value_position(df):

    #Getting the average price per share by taking mean of all values in the columns
    avg_df = df.groupby("Symbol").mean()
    
    return avg_df

In [ ]:
#Passing in the equities dataframe to get avg value for each symbol
avg_price_df = get_avg_value_position(inlaw_equities_df)

#Reset index column so we can compare with other dataframes using same index
avg_price_df = avg_price_df.reset_index()

In [ ]:
#Passing in the equities dataframe to get total position for each symbol
equities_positions_df = get_total_equities_position(inlaw_equities_df)

#Reset index column so we can compare with other dataframes using same index
equities_positions_df = equities_positions_df.reset_index()

#Change Symbols to a list of strings so it can be passed into alpaca api call as a parameter
symbol_list = equities_positions_df['Symbol'].tolist()
symbol_list_of_strings = [''.join(ele) for ele in symbol_list]
symbol_list_of_strings

In [ ]:
# Format current date as ISO format
start_date = pd.Timestamp("2021-05-19", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2021-05-19", tz="America/New_York").isoformat()

# Set the tickers parameter using the list of symbols
tickers = symbol_list_of_strings

# Set timeframe to one day ('1D') for the Alpaca API
timeframe = "1D"

# Get current closing prices for all symbols in the equities dataframe
equities_portfolio_df = alpaca.get_barset(
    tickers,
    timeframe,
    start = start_date,
    end = end_date
).df

#Reset index column so we can compare with other dataframes using same index
equities_portfolio_df = equities_portfolio_df.reset_index()

#Dropping time column so we can create a matching index column
equities_portfolio_df = equities_portfolio_df.drop(columns='time')

In [ ]:
# Create an empty `closing_prices_df` DataFrame using Pandas
closing_prices_df = pd.DataFrame()

# Populate the `closing_prices_df` DataFrame by accessing the `close` column
# from the `equities_portfolio_df` DataFrame for both all Symbols.
closing_prices_df["ABBV"] = equities_portfolio_df["ABBV"]['close']
closing_prices_df["AMD"] = equities_portfolio_df["AMD"]["close"]
closing_prices_df["ETAC"] = equities_portfolio_df["ETAC"]["close"]
closing_prices_df["HEC"] = equities_portfolio_df["HEC"]["close"]
closing_prices_df["PSTH"] = equities_portfolio_df["PSTH"]["close"]
closing_prices_df["SPY"] = equities_portfolio_df["SPY"]["close"]

#Transpose the dataframe so we can group by symbol.
closing_prices_df = closing_prices_df.T

#Reset index so we can match to other dataframes with same index column. 
closing_prices_df = closing_prices_df.reset_index()

#Change name of column to "Symbol" and "close"
closing_prices_df = closing_prices_df.rename({'index': 'Symbol', 0: 'close'}, errors='raise', axis=1)

In [ ]:
#Replace 'Average Price' column in 'equities_positions_df' with 'Average Price' column from 'avg_price_df'
equities_positions_df['Average Price'] = avg_price_df['Average Price']

#Adding columns to 'equities_positions_df' 
equities_positions_df['Total Cost Basis'] = equities_positions_df['Total Position'] * equities_positions_df["Average Price"]
equities_positions_df['Total Current Value'] = equities_positions_df['Total Position'] * closing_prices_df["close"]
equities_positions_df['Gain/Loss'] = equities_positions_df['Total Current Value'] - equities_positions_df["Total Cost Basis"]
equities_positions_df['% Gain/Loss'] = equities_positions_df['Gain/Loss'] / equities_positions_df["Total Cost Basis"]
equities_positions_df['Gain/Loss per Share'] = equities_positions_df['Gain/Loss'] / equities_positions_df["Total Position"]
equities_positions_df['Current Share Price'] = closing_prices_df['close']

In [ ]:
#Changing datetime format for inlaw_df
#inlaw_df['Date'] = pd.to_datetime(inlaw_df['Date']).dt.date

In [ ]:
equities_positions_df.hvplot.bar(title="Gain/Loss in % for Equities Portfolio", x="Symbol", y='% Gain/Loss')

In [ ]:
equities_positions_df.hvplot.bar(title="Gain/Loss in $ for Equities Portfolio", x="Symbol", y='Gain/Loss', ylabel="Gain/Loss in $")

In [ ]:
equities_positions_df.plot.pie(title="Total Portfolio Value Makeup by Symbol", y="Total Current Value", labels=["ABBV", "AMD", "ETAC", "HEC", "PSTH", "SPY"])

In [ ]:
equities_positions_df.hvplot.bar(title="Current Share Price vs. Average Cost Per Share", y='Current Share Price', x='Symbol', color="blue") * equities_positions_df.hvplot.bar(y='Average Price', x='Symbol', color='red') 

In [ ]:
def short_put_gross_net_value(df):

    # Filter the dataframe by the short actions
    df = df.loc[(df["Action"] == "SELL_TO_OPEN") | (df["Action"] == "SELL_TO_CLOSE")]
    df = df.loc[df["Call or Put"] == "PUT"]

   
    #Create variables to hold the gross value, total commission fees, and total fees to help calculate the net value
    gross_value = df["Value"].sum()
    total_commissions = df["Commissions"].sum()
    total_fees = df["Fees"].sum()
    net_value = gross_value + total_commissions + total_fees
    
    return df

In [ ]:
short_put_df = short_put_gross_net_value(inlaw_df)
short_put_df = short_put_df.groupby('Underlying Symbol').sum()

In [ ]:
short_put_plot = short_put_df.hvplot.bar(x='Underlying Symbol', y="Value", rot=90, title="Short Put Gross Values")
short_put_plot

In [ ]:
def long_put_gross_net_value(df):

    #Filter the dataframe by the long actions
    df = df.loc[(df["Action"] == "BUY_TO_OPEN") | (df["Action"] == "BUY_TO_CLOSE")]
    df = df.loc[df["Call or Put"] == "PUT"]
    
    return df

In [ ]:
long_put_df = long_put_gross_net_value(inlaw_df)
long_put_df = long_put_df.groupby('Underlying Symbol').sum() * -1

In [ ]:
long_put_plot = long_put_df.hvplot.bar(x='Underlying Symbol', y="Value", rot=90, title="Long Put Gross Values")
long_put_plot

In [ ]:
def short_call_gross_net_value(df):

    #Filter the dataframe by the short actions
    df = df.loc[(df["Action"] == "SELL_TO_OPEN") | (df["Action"] == "SELL_TO_CLOSE")]
    df = df.loc[df["Call or Put"] == "CALL"]
    
    return df

In [ ]:
short_call_df = short_call_gross_net_value(inlaw_df)
short_call_df = short_call_df.groupby('Underlying Symbol').sum() 

In [ ]:
short_call_plot = short_call_df.hvplot.bar(x='Underlying Symbol', y="Value", rot=90, title="Short Call Gross Values")
short_call_plot

In [ ]:
def long_call_gross_net_value(df):

    #Filter the dataframe by the long actions
    df = df.loc[(df["Action"] == "BUY_TO_OPEN") | (df["Action"] == "BUY_TO_CLOSE")]
    df = df.loc[df["Call or Put"] == "CALL"]
    
    return df

In [ ]:
long_call_df = long_call_gross_net_value(inlaw_df)
long_call_df = long_call_df.groupby('Underlying Symbol').sum() * -1

In [ ]:
long_call_plot = long_call_df.hvplot.bar(x='Underlying Symbol', y="Value", rot=90, title="Long Call Gross Values")
long_call_plot

In [ ]:
combined_put_plot = short_put_plot * long_put_plot
combined_put_plot.opts(title="Short & Long Put Values")

In [ ]:
combined_call_plot = short_call_plot * long_call_plot
combined_call_plot.opts(title="Short & Long Call Values")

In [ ]:
#Changing datetime format for inlaw_df
#inlaw_equities_df['Date'] = pd.to_datetime(inlaw_equities_df['Date']).dt.date

In [ ]:
# Format current date as ISO format
#start_date = pd.Timestamp("2020-03-15", tz="America/New_York").isoformat()
#end_date = pd.Timestamp("2021-04-23", tz="America/New_York").isoformat()

# Set the tickers parameter using the list of symbols
#tickers = symbol_list_of_strings

# Set timeframe to one day ('1D') for the Alpaca API
#timeframe = "1D"

# Get current closing prices for all symbols in the equities dataframe
#equities_closing_prices_df = alpaca.get_barset(
#    tickers,
 #   timeframe,
#    start = start_date,
#    end = end_date
#).df
#equities_closing_prices_df = equities_closing_prices_df.reset_index()
#equities_closing_prices_df = (
#    equities_closing_prices_df
#    .groupby('ABBV', 'AMD', 'ETAC', 'HEC', 'PSTH', 'SPY')
#    .sum()
#)

In [ ]:
#inlaw_equities_df.hvplot(x="Date", y="Average Price", groupby='Symbol', rot=45) * equities_closing_prices_df.hvplot(x='time', y='close')

In [ ]:
#equities_closing_prices_df.plot(group)

In [ ]:
#def profit_per_trade(Value, Commissions, Fees):
    #return (Value + Commissions + Fees)

#profit_per_equity_trade_df = profit_per_trade(inlaw_equities_df['Value'], inlaw_equities_df['Commissions'], inlaw_equities_df['Fees'])

In [ ]:
#profit_p_trade = profit_per_option_trade_df.cumsum() * -1
#profit_per_equity_trade_df

In [ ]:
#profit_per_option_trade_df.hvplot(x="")

In [ ]:
query = """
SELECT *
FROM inlaw
WHERE Action = "SELL_TO_OPEN" 
OR Action = "SELL_TO_CLOSE";
"""
SELL_TO_OPEN_OR_CLOSE_df = pd.read_sql_query(query, con=engine)


In [ ]:
#SELL_TO_OPEN_OR_CLOSE_df.plot(x='Date', y="Average Price")

In [ ]:
query = """
SELECT *
FROM inlaw
WHERE Action = "BUY_TO_CLOSE" 
OR Action = "BUY_TO_OPEN";
"""
BUY_TO_OPEN_OR_CLOSE_df = pd.read_sql_query(query, con=engine)

In [ ]:
query = """
SELECT *
FROM inlaw
WHERE "Call or Put" = "CALL";
"""
CALL_df = pd.read_sql_query(query, con=engine)

In [ ]:
query = """
SELECT *
FROM inlaw
WHERE "Call or Put" = "PUT";
"""
PUT_df = pd.read_sql_query(query, con=engine)

In [ ]:
query = """
SELECT *
FROM inlaw
WHERE "Instrument Type" = "Equity Option";
"""
equity_option_df = pd.read_sql_query(query, con=engine)

In [ ]:
query = """
SELECT *
FROM inlaw
WHERE "Instrument Type" = "Future Option";
"""
future_option_df = pd.read_sql_query(query, con=engine)

In [ ]:
query = """
SELECT *
FROM inlaw
WHERE "Instrument Type" = "Future Option" 
OR "Instrument Type" = "Equity Option";
"""
option_df = pd.read_sql_query(query, con=engine)
option_df.to_sql('option', engine, if_exists='replace')

In [ ]:
query = """
SELECT *
FROM option
WHERE Action = "SELL_TO_OPEN" 
OR Action = "SELL_TO_CLOSE";
"""
sell_option_df = pd.read_sql_query(query, con=engine)

In [ ]:
query = """
SELECT *
FROM inlaw
WHERE Action = "BUY_TO_OPEN" 
OR Action = "BUY_TO_CLOSE";
"""
buy_option_df = pd.read_sql_query(query, con=engine)

In [ ]:
query = """
SELECT *
FROM inlaw
WHERE "Instrument Type" = "Equity";
"""
equity_df = pd.read_sql_query(query, con=engine)

In [ ]:
profit_per_equity_trade_df = profit_per_trade(equity_df['Value'], equity_df['Commissions'], equity_df['Fees'])
profit_per_equity_trade_df.cumsum()

In [ ]:
query = """
SELECT "Order #",
    Sum(Value) AS Value
FROM inlaw
GROUP BY "Order #"
"""
order_sorted = pd.read_sql_query(query, con=engine)

In [ ]:
order_sorted['Value'].describe()

In [ ]:
order_sorted['Value'].cumsum()

In [ ]:
order_sorted['Value'].sum()

In [ ]:
order_sorted['Value'].cumsum().plot()

In [ ]:
query = """
SELECT "Underlying Symbol",
    Sum(Value) AS Value
FROM inlaw
GROUP BY "Underlying Symbol"
"""
ticker_sorted = pd.read_sql_query(query, con=engine)

In [ ]:
query = """
SELECT *
FROM open_df
WHERE "Expiration Date" = "Expiration Date" 
AND "Strike Price" = "Strike Price"
AND "Call or Put" = "Call or Put"
AND "Order #" = "Order #"; add SS Open to column Strategy
"""
SELL_TO_OPEN_OR_CLOSE_df = pd.read_sql_query(query, con=engine)

In [ ]:
query = """
SELECT *
FROM close_df
WHERE "Expiration Date" = "Expiration Date" 
AND "Strike Price" = "Strike Price"
AND "Call or Put" = "Call or Put"; add SS Close
"""
SELL_TO_OPEN_OR_CLOSE_df = pd.read_sql_query(query, con=engine)